# Setting up the model

This section imports the required packages and downloads the data used for training.

In [1]:
# Imports and setup
import os
import torch
import torch.nn as nn
from torch.nn import functional as F

# import matplotlib.pyplot as plt

Importing copy of required datasets from personal Github repository if not found in current directory.

In [2]:
# Download the ZIP file using wget
ZIP_URL = 'https://github.com/tpchikumbu/GPTiny/archive/main.zip'
PROJECT_DIR = os.getcwd() + '/LMDatasets'
print('Searching for data in ', PROJECT_DIR)

if not os.path.isdir(PROJECT_DIR):

  !wget -O "GPTiny-main.zip" "$ZIP_URL"
  # Extract only the specific folder from the ZIP file
  !unzip -q "GPTiny-main.zip" "GPTiny-main/LMDatasets/*" -d "."
  !mv "GPTiny-main/LMDatasets" "."

  # Remove temporary files
  !rm -rf "GPTiny-main.zip"
  !rm -rf "GPTiny-main"

else:
  print('Data already downloaded')

Searching for data in  /home/peter/Documents/Hons/NLP/GPTiny/LMDatasets
Data already downloaded


Run to choose a language for the datasets and read them into environment

In [3]:
language = "nr" # Options include nr, ss, xh, zu
file_name = f'{PROJECT_DIR}/nchlt_text.{language}'

# Load data
with open(file_name + ".train", 'r', encoding='utf-8') as f:
    train_df = f.read()
with open(file_name + ".valid", 'r', encoding='utf-8') as f:
    dev_df = f.read()
with open(file_name + ".test", 'r', encoding='utf-8') as f:
    test_df = f.read()

print('Train shape: ', len(train_df))
print('Dev shape: ', len(dev_df))
print('Test shape: ', len(test_df))

Train shape:  6382803
Dev shape:  441906
Test shape:  444199
